In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

In [2]:
games = pd.read_csv("data/games.csv")
players = pd.read_csv("data/players.csv")
plays = pd.read_csv("data/plays.csv")
PFFScouting = pd.read_csv('data/PFFScoutingData.csv')

In [3]:
#compress size for saving memory space
def downcast(df, verbose=True):
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        dtype_name = df[col].dtype.name
        if dtype_name == 'object':
            pass
        elif dtype_name == 'bool':
            df[col] = df[col].astype('int8')
        elif dtype_name.startswith('int') or (df[col].round() == df[col]).all():
            df[col] = pd.to_numeric(df[col], downcast='integer')
        else:
            df[col] = pd.to_numeric(df[col], downcast='float')
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose:
        print('{:.1f}% Compressed'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df
def resumetable(df):
    print(f'Shape : {df.shape}')
    summary = pd.DataFrame(df.dtypes, columns=['Data Type'])
    summary = summary.reset_index()
    summary = summary.rename(columns={'index': 'Feature'})
    summary['Num of null'] = df.isnull().sum().values
    summary['Num of unique'] = df.nunique().values
    summary['First value'] = df.loc[0].values
    summary['Second value'] = df.loc[1].values
    summary['Third value'] = df.loc[2].values
    return summary

In [4]:
games = downcast(games)
players = downcast(players)
plays = downcast(plays)
PFFScouting = downcast(PFFScouting)

30.3% Compressed
17.8% Compressed
43.0% Compressed
13.7% Compressed


In [5]:
#check data
resumetable(players)
resumetable(games)
resumetable(plays)
resumetable(PFFScouting)

Shape : (2732, 7)
Shape : (764, 7)
Shape : (19979, 25)
Shape : (19979, 20)


,Feature,Data Type,Num of null,Num of unique,First value,Second value,Third value
0,gameId,int32,0,764,2018090600,2018090600,2018090600
1,playId,int16,0,4435,37,366,658
2,snapDetail,object,14060,5,NaN,OK,NaN
3,snapTime,float32,14061,58,NaN,0.84,NaN
4,operationTime,float32,14061,97,NaN,2.12,NaN
5,hangTime,float32,6881,296,3.85,4.46,NaN
6,kickType,object,6256,11,D,N,NaN
7,kickDirectionIntended,object,6278,3,R,C,NaN
8,kickDirectionActual,object,6278,3,R,C,NaN
9,returnDirectionIntended,object,15225,3,NaN,C,NaN


In [6]:
#convert height & weight
players_heights = players["height"] # Get the Height data from DataFrame
players_heights = players_heights.apply(lambda x: x.split("-")) # Split the heights by hyphen ("-")

# Convert Heights to Centimeters and add them to DataFrame
players["height"] = players_heights.apply(lambda x: int(x[0]) * 12 + int(x[1]) if len(x) == 2 else int(x[0])) * 2.54

# Convert Weights to Kilograms and them to DataFrame
players["weight"] = round(players.weight * 0.453592, 2)

In [7]:
#fill in NAN Value on age and college name (only players in special team)
players.loc[players['displayName'] =='Hunter Niswander', ['birthDate']] = '1994-11-26'
players.loc[players['displayName'] =='Taylor Russolino', ['birthDate']] = '1989-05-23'
players.loc[players['displayName'] =='Brandon Wright', ['collegeName']] = 'North Carolina State'
players.loc[players['displayName'] =='Hunter Niswander', ['collegeName']] = 'Northwestern'
players.loc[players['displayName'] =='Taylor Russolino', ['collegeName']] = 'Mississippi'

In [8]:
#convert birthdate to age
from datetime import date
players['birthDate'] = pd.to_datetime(players['birthDate'])
today = pd.to_datetime("today")
players["age"]=today.year - players["birthDate"].dt.year
players

,nflId,height,weight,birthDate,collegeName,Position,displayName,age
0,42901,185.42,94.35,1992-07-25,James Madison,SS,Dean Marlowe,29.0
1,43501,182.88,99.79,1994-08-11,Central Michigan,FS,Kavon Frazier,27.0
2,43534,182.88,92.99,1993-06-02,Illinois,SS,Clayton Fejedelem,28.0
3,43535,185.42,106.59,1992-12-22,Temple,MLB,Tyler Matakevich,29.0
4,44174,185.42,107.05,1993-02-22,Texas Tech,ILB,Sam Eguavoen,28.0
...,...,...,...,...,...,...,...,...
2727,33216,190.50,145.15,1986-06-16,Central Florida,G,Josh Sitton,35.0
2728,33184,190.50,120.20,1985-05-02,Winston-Salem State,DE,William Hayes,36.0
2729,45651,175.26,89.81,1993-06-16,Iowa,WR,Riley McCarron,28.0
2730,41284,185.42,104.33,1992-10-20,LSU,RB,Jeremy Hill,29.0


In [9]:
#filter playtype: punt/field goal
plays= plays.loc[(plays['specialTeamsPlayType'] == 'Field Goal') | (plays['specialTeamsPlayType'] == 'Punt' )]

In [10]:
plays

,gameId,playId,playDescription,quarter,down,yardsToGo,possessionTeam,specialTeamsPlayType,specialTeamsResult,kickerId,...,penaltyCodes,penaltyJerseyNumbers,penaltyYards,preSnapHomeScore,preSnapVisitorScore,passResult,kickLength,kickReturnYardage,playResult,absoluteYardlineNumber
1,2018090600,366,"(9:20) C.Johnston punts 56 yards to ATL 36, Ce...",1,4,4,PHI,Punt,Return,45603.0,...,UNSd,PHI 18,-15.0,0,0,NaN,56.0,5.0,36,18
2,2018090600,658,"(5:03) M.Bryant 21 yard field goal is GOOD, Ce...",1,4,3,ATL,Field Goal,Kick Attempt Good,27091.0,...,NaN,NaN,NaN,0,0,NaN,21.0,NaN,0,13
4,2018090600,872,"(:33) C.Johnston punts 65 yards to end zone, C...",1,4,18,PHI,Punt,Touchback,45603.0,...,NaN,NaN,NaN,0,3,NaN,65.0,NaN,45,45
5,2018090600,973,"(14:18) M.Bosher punts 49 yards to PHI 40, Cen...",2,4,19,ATL,Punt,Fair Catch,37267.0,...,NaN,NaN,NaN,0,3,NaN,49.0,NaN,49,21
6,2018090600,1368,"(6:12) J.Elliott 26 yard field goal is GOOD, C...",2,4,8,PHI,Field Goal,Kick Attempt Good,44966.0,...,NaN,NaN,NaN,0,3,NaN,26.0,NaN,0,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19962,2021010315,2121,"(9:36) T.Vizcaino 47 yard field goal is GOOD, ...",3,4,8,SF,Field Goal,Kick Attempt Good,47590.0,...,NaN,NaN,NaN,3,6,NaN,47.0,NaN,0,39
19964,2021010315,2235,"(8:39) M.Dickson punts 43 yards to SF 30, Cent...",3,4,8,SEA,Punt,Return,46218.0,...,NaN,NaN,NaN,6,6,NaN,43.0,21.0,22,37
19965,2021010315,2366,"(6:23) T.Vizcaino 33 yard field goal is GOOD, ...",3,4,15,SF,Field Goal,Kick Attempt Good,47590.0,...,NaN,NaN,NaN,6,6,NaN,33.0,NaN,0,25
19967,2021010315,2497,"(5:17) M.Dickson punts 52 yards to SF 22, Cent...",3,4,9,SEA,Punt,Return,46218.0,...,NaN,NaN,NaN,9,6,NaN,52.0,5.0,47,36


In [11]:
#drop row that have kicktype but don't have kick direction
PFFScouting=PFFScouting.drop(PFFScouting[(PFFScouting['kickType'].notnull()) & (PFFScouting['kickDirectionIntended'].isna())].index)

In [12]:
#merge all file (except tracking)
playgames=pd.merge(plays,games,on='gameId',how='left')
playgamesScout=pd.merge(playgames,PFFScouting,on=['gameId','playId'],how='left')
alldata= pd.merge(playgamesScout,players, left_on=['kickerId'],right_on=['nflId'],how='left')

In [13]:
#replace short value with full value
alldata['Position']= alldata['Position'].replace(['P','K'],['Punter','Kicker'])
alldata['kickType']= alldata['kickType'].replace(['D','F','K','O','P','Q','S','B','N','R','A'],['Deep','Flat','Free Kick','Obvious Onside','Pooch Kick','Squib','Surprise Onside','Deep Direct OOB','Normal','Rugby style punt','Aussie-style punts'])
alldata['kickDirectionIntended']= alldata['kickDirectionIntended'].replace(['L','R','C'],['Left','Right','Center'])
alldata['kickDirectionActual']= alldata['kickDirectionActual'].replace(['L','R','C'],['Left','Right','Center'])
alldata['returnDirectionIntended']= alldata['returnDirectionIntended'].replace(['L','R','C'],['Left','Right','Center'])
alldata['returnDirectionActual']= alldata['returnDirectionActual'].replace(['L','R','C'],['Left','Right','Center'])
alldata['kickContactType']= alldata['kickContactType'].replace(['BB', 'BC', 'BF', 'BOG', 'CC', 'CFFG', 'DEZ', 'ICC', 'KTB', 'KTC', 'KTF', 'MBC', 'MBDR', 'OOB'],['Bounced Backwards', 'Bobbled Catch from Air', 'Bounced Forwards', 'Bobbled on Ground', 'Clean Catch from Air', 'Clean Field From Ground', 'Direct to Endzone', 'Incidental Coverage Team Contact', 'Kick Team Knocked Back', 'Kick Team Catch', 'Kick Team Knocked Forward', 'Muffed by Contact with Non-Designated Returner', 'Muffed by Designated Returner', 'Directly Out Of Bounds'])
alldata['snapDetail']= alldata['snapDetail'].replace(['<','>','H','L','OK'],['Left','Right','High','Low','Accurate snap'])
alldata['passResult']= alldata['passResult'].replace(['C','I','S','IN','R'],['Complete pass','Incomplete pass','Quarterback sack','Intercepted pass','Scramble'])

In [14]:
alldata

,gameId,playId,playDescription,quarter,down,yardsToGo,possessionTeam,specialTeamsPlayType,specialTeamsResult,kickerId,...,vises,kickContactType,nflId,height,weight,birthDate,collegeName,Position,displayName,age
0,2018090600,366,"(9:20) C.Johnston punts 56 yards to ATL 36, Ce...",1,4,4,PHI,Punt,Return,45603.0,...,ATL 83; ATL 27; ATL 34; ATL 21,Clean Catch from Air,45603.0,180.34,88.00,1992-02-24,Ohio State,Punter,Cameron Johnston,29.0
1,2018090600,658,"(5:03) M.Bryant 21 yard field goal is GOOD, Ce...",1,4,3,ATL,Field Goal,Kick Attempt Good,27091.0,...,NaN,NaN,27091.0,175.26,92.08,1975-05-29,Baylor,Kicker,Matt Bryant,46.0
2,2018090600,872,"(:33) C.Johnston punts 65 yards to end zone, C...",1,4,18,PHI,Punt,Touchback,45603.0,...,ATL 83; ATL 34; ATL 21,Bounced Forwards,45603.0,180.34,88.00,1992-02-24,Ohio State,Punter,Cameron Johnston,29.0
3,2018090600,973,"(14:18) M.Bosher punts 49 yards to PHI 40, Cen...",2,4,19,ATL,Punt,Fair Catch,37267.0,...,PHI 29; PHI 32,Clean Catch from Air,37267.0,182.88,94.35,1987-10-18,Miami (Fla.),Punter,Matt Bosher,34.0
4,2018090600,1368,"(6:12) J.Elliott 26 yard field goal is GOOD, C...",2,4,8,PHI,Field Goal,Kick Attempt Good,44966.0,...,NaN,NaN,44966.0,175.26,75.75,1995-01-21,Memphis,Kicker,Jake Elliott,26.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8643,2021010315,2121,"(9:36) T.Vizcaino 47 yard field goal is GOOD, ...",3,4,8,SF,Field Goal,Kick Attempt Good,47590.0,...,NaN,NaN,47590.0,187.96,92.99,1996-07-31,Washington,Kicker,Tristan Vizcaino,25.0
8644,2021010315,2235,"(8:39) M.Dickson punts 43 yards to SF 30, Cent...",3,4,8,SEA,Punt,Return,46218.0,...,SF 17; SF 41,Clean Catch from Air,46218.0,187.96,94.35,1996-01-04,Texas,Punter,Michael Dickson,25.0
8645,2021010315,2366,"(6:23) T.Vizcaino 33 yard field goal is GOOD, ...",3,4,15,SF,Field Goal,Kick Attempt Good,47590.0,...,NaN,NaN,47590.0,187.96,92.99,1996-07-31,Washington,Kicker,Tristan Vizcaino,25.0
8646,2021010315,2497,"(5:17) M.Dickson punts 52 yards to SF 22, Cent...",3,4,9,SEA,Punt,Return,46218.0,...,SF 17; SF 41,Clean Catch from Air,46218.0,187.96,94.35,1996-01-04,Texas,Punter,Michael Dickson,25.0


In [15]:
#analyse possibility of success field goal as kick length
Plays_FG_Kickatt = plays.loc[(plays['specialTeamsPlayType'] == 'Field Goal') & ((plays['specialTeamsResult'] == 'Kick Attempt Good') | (plays['specialTeamsResult'] == 'Kick Attempt No Good' ))]

In [16]:
lambda_func = lambda Plays_FG_Kickatt: pd.Series({'KickLength<30': sum(Plays_FG_Kickatt.kickLength < 30)})
kicklength_interval=Plays_FG_Kickatt.groupby('gameId').apply(lambda_func)
lambda_func = lambda Plays_FG_Kickatt: pd.Series({'KickLength<30': sum((Plays_FG_Kickatt.kickLength < 30)&(Plays_FG_Kickatt.specialTeamsResult
=='Kick Attempt Good'))})
kicklength_interval['KickLength<30succ']=Plays_FG_Kickatt.groupby('gameId').apply(lambda_func)

In [17]:
lambda_func = lambda Plays_FG_Kickatt: pd.Series({'KickLength30-39': sum((Plays_FG_Kickatt.kickLength >29)& (Plays_FG_Kickatt.kickLength<40))})
kicklength_interval["KickLength30-39"]=Plays_FG_Kickatt.groupby('gameId').apply(lambda_func)
lambda_func = lambda Plays_FG_Kickatt: pd.Series({'KickLength30-39': sum((Plays_FG_Kickatt.kickLength >29)& (Plays_FG_Kickatt.kickLength<40)&(Plays_FG_Kickatt.specialTeamsResult
=='Kick Attempt Good'))})
kicklength_interval["KickLength30-39succ"]=Plays_FG_Kickatt.groupby('gameId').apply(lambda_func)

In [18]:
lambda_func = lambda Plays_FG_Kickatt: pd.Series({'KickLength40-49': sum((Plays_FG_Kickatt.kickLength >39)& (Plays_FG_Kickatt.kickLength<50))})
kicklength_interval["KickLength40-49"]=Plays_FG_Kickatt.groupby('gameId').apply(lambda_func)
lambda_func = lambda Plays_FG_Kickatt: pd.Series({'KickLength40-49': sum((Plays_FG_Kickatt.kickLength >39)& (Plays_FG_Kickatt.kickLength<50)&(Plays_FG_Kickatt.specialTeamsResult
=='Kick Attempt Good'))})
kicklength_interval["KickLength40-49succ"]=Plays_FG_Kickatt.groupby('gameId').apply(lambda_func)

In [19]:
lambda_func = lambda Plays_FG_Kickatt: pd.Series({'KickLength50-59': sum((Plays_FG_Kickatt.kickLength >49)& (Plays_FG_Kickatt.kickLength<60))})
kicklength_interval["KickLength50-59"]=Plays_FG_Kickatt.groupby('gameId').apply(lambda_func)
lambda_func = lambda Plays_FG_Kickatt: pd.Series({'KickLength30-39': sum((Plays_FG_Kickatt.kickLength > 49)& (Plays_FG_Kickatt.kickLength<60)&(Plays_FG_Kickatt.specialTeamsResult
=='Kick Attempt Good'))})
kicklength_interval["KickLength50-59succ"]=Plays_FG_Kickatt.groupby('gameId').apply(lambda_func)

In [20]:
lambda_func = lambda Plays_FG_Kickatt: pd.Series({'KickLength60-69': sum((Plays_FG_Kickatt.kickLength >59)& (Plays_FG_Kickatt.kickLength<70))})
kicklength_interval["KickLength60-69"]=Plays_FG_Kickatt.groupby('gameId').apply(lambda_func)
lambda_func = lambda Plays_FG_Kickatt: pd.Series({'KickLength30-39': sum((Plays_FG_Kickatt.kickLength >59)& (Plays_FG_Kickatt.kickLength<70)&(Plays_FG_Kickatt.specialTeamsResult
=='Kick Attempt Good'))})
kicklength_interval["KickLength60-69succ"]=Plays_FG_Kickatt.groupby('gameId').apply(lambda_func)

In [21]:
kicklength_interval.fillna(0)

,KickLength<30,KickLength<30succ,KickLength30-39,KickLength30-39succ,KickLength40-49,KickLength40-49succ,KickLength50-59,KickLength50-59succ,KickLength60-69,KickLength60-69succ
gameId,,,,,,,,,,
2018090600,2,2,0,0,0,0,1,1,0,0
2018090900,0,0,2,2,1,1,1,0,0,0
2018090901,0,0,0,0,1,0,0,0,0,0
2018090902,1,1,2,2,1,1,2,1,0,0
2018090903,1,1,1,1,1,1,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...
2021010311,2,2,2,2,0,0,1,0,0,0
2021010312,2,2,1,1,2,2,1,1,0,0
2021010313,2,2,0,0,1,0,1,0,0,0


In [22]:
kicklengthh=kicklength_interval.sum(axis=0).to_frame()

In [23]:
kicklengthh.reset_index()
kicklengthh=kicklengthh.transpose()

In [24]:
kicklengthh["%KickLength<30"]=kicklengthh["KickLength<30succ"]/kicklengthh["KickLength<30"]*100
kicklengthh["%KickLength30-39"]=kicklengthh["KickLength30-39succ"]/kicklengthh["KickLength30-39"]*100
kicklengthh["%KickLength40-49"]=kicklengthh["KickLength40-49succ"]/kicklengthh["KickLength40-49"]*100
kicklengthh["%KickLength50-59"]=kicklengthh["KickLength50-59succ"]/kicklengthh["KickLength50-59"]*100
kicklengthh["%KickLength60-69"]=kicklengthh["KickLength60-69succ"]/kicklengthh["KickLength60-69"]*100
kicklengthh = kicklengthh.drop(kicklengthh.columns[[0, 1, 2,3,4,5,6,7,8,9]], axis=1) 

In [25]:
#possibility of success field goal as kick length
kicklengthh

,%KickLength<30,%KickLength30-39,%KickLength40-49,%KickLength50-59,%KickLength60-69
0,98.242812,93.125811,77.832512,65.263158,33.333333
